In [1]:
import time
from IPython.display import clear_output
from STservo import ST3215, Calibrator

In [ ]:
dev_id=[7, 8, 9, 10, 11, 12]
calib = Calibrator(port='COM17', dev_id=dev_id, __set_posi_corr__=True)
try:
    while True:
        time.sleep(0.3)
        clear_output(wait=True)
        calib.update()
except KeyboardInterrupt:
    clear_output(wait=True)
    config = calib.config()

config

In [ ]:
for _id, _c in config.items():
    calib.__set_posi_corr__(dev_id=_id, step=_c['corr'])

time.sleep(1)
try:
    while True:
        time.sleep(0.3)
        clear_output(wait=True)
        for _id in dev_id:
            status, corr = calib.__get_posi_corr__(dev_id=_id)
            print(f'id: {_id}, corr: {corr}, posi: {calib.readPosi(dev_id=[_id], debug=True)[_id]["posi"]}')  
except KeyboardInterrupt:
    pass

In [2]:
servo_bus = ST3215(port='COM17')

In [4]:
servo_bus.move2Posi(dev_id=7, posi=2500, velo=1800, acc=100)


In [ ]:
servo_bus.__set_torque_mode__(7, 'torque')

In [ ]:
servo_bus.readPosi(dev_id=7)

In [ ]:
servo_bus.ping(1)

In [ ]:
posi = 0x0FFF
servo_bus.move2Posi(dev_id=4, posi=posi, velo=1800, acc=100)
servo_bus.move2Posi(dev_id=3, posi=posi, velo=1800, acc=100)
time.sleep(1)
print('posi', servo_bus.readPosi(dev_id=[3, 4], debug=True))
time.sleep(6)
print('posi', servo_bus.readPosi(dev_id=[3, 4], debug=True))
time.sleep(3)
servo_bus.move2Posi(dev_id=[3, 4], posi=[0, 0], velo=1800, acc=100)

In [ ]:
servo.__set_torque_mode__(dev_id=3, mode='torque')
servo.__set_posi_corr__(dev_id=3, step=2000)
servo.move2Posi(dev_id=[3], posi=[1000], velo=1800, acc=100)

In [ ]:
servo.move2Posi(dev_id=[3, 4], posi=[2000, 3500], velo=1800, acc=100)

In [ ]:
status, posi_corr = servo.__get_posi_corr__(dev_id=4)
posi_corr

In [ ]:
servo.__set_torque_mode__(dev_id=3, mode='free')

In [ ]:
t = time.time()
while True:
    posi = servo.readPosi(dev_id=[3], debug=True)[3]['posi']
    posi += 2500
    servo.move2Posi(dev_id=[4], posi=[posi], velo=1800, acc=100)
    time.sleep(0.1)
    if time.time() - t > 1:
        t = time.time()
        print(posi-2500, posi)